In [ ]:
import os
import torch
import sys
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# --- CONFIGURATION ---
# Remplacez par votre clé API réelle
os.environ["GROQ_API_KEY"]=os.getenv("Groq_API_KEY")

# Chemins
DATA_PATH = "data/"            # Mettez vos PDFs dans ce dossier
DB_FAISS_PATH = "vectorstore/db_faiss"

# Modèles Optimisés
# 1. Embeddings MULTILINGUES (Comprend FR <-> EN)
MODEL_EMBEDDING = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
# 2. LLM Performant
MODEL_LLM = "llama-3.3-70b-versatile"

# Détection GPU
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚙️ Configuration : Embeddings sur {DEVICE.upper()} | Modèle : {MODEL_LLM}")

⚙️ Configuration : Embeddings sur CUDA | Modèle : llama-3.3-70b-versatile


In [8]:
import os
import re
import torch
import sys

# --- CORRECTION DES IMPORTS ---
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# C'est ici que ça changeait : on utilise langchain_core maintenant
from langchain_core.documents import Document 

# --- 1. CONFIGURATION ---
DATA_PATH = "data/raw"            # Assurez-vous que vos 5 PDFs sont dans ce dossier
DB_FAISS_PATH = "vectorstore/db_faiss"

# Modèle Multilingue (Arabe + Français + Anglais)
MODEL_EMBEDDING = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def clean_text(text):
    """
    Fonction de nettoyage avancé pour retirer le bruit des PDF.
    """
    if not text: return ""

    # 1. Retirer les numéros de pages isolés
    text = re.sub(r'Page\s+\d+|^\d+\s*$', '', text, flags=re.MULTILINE)
    text = re.sub(r'\d+\s+\|\s+P\s+a\s+g\s+e', '', text) # Cas spécifique Wheeling
    
    # 2. Retirer les en-têtes répétitifs
    text = re.sub(r'Undergraduate\s+Catalog\s+2024-2025', '', text, flags=re.IGNORECASE)
    
    # 3. Réparer les césures de mots (hyphenation)
    text = re.sub(r'(\w+)-\s+(\w+)', r'\1\2', text)
    
    # 4. Supprimer les sauts de ligne multiples
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    return text.strip()

def load_and_process_documents():
    print(f"--- 🚀 Démarrage du Traitement Avancé (Sur {DEVICE.upper()}) ---")
    
    # Vérification du dossier
    if not os.path.exists(DATA_PATH):
        os.makedirs(DATA_PATH)
        print(f"⚠️  Le dossier '{DATA_PATH}' a été créé. Veuillez y déposer vos PDF et relancer.")
        return

    # 1. Chargement Brut
    loader = PyPDFDirectoryLoader(DATA_PATH)
    raw_docs = loader.load()
    print(f"📄 {len(raw_docs)} pages brutes chargées.")

    if not raw_docs:
        print(f"❌ Erreur : Le dossier '{DATA_PATH}' est vide. Ajoutez vos PDF.")
        return

    processed_docs = []
    
    # 2. Nettoyage et Injection de Contexte
    print("🧹 Nettoyage et Enrichissement des données...")
    for doc in raw_docs:
        # Identification de la source
        full_source = doc.metadata.get('source', '')
        filename = os.path.basename(full_source) # Extrait juste le nom du fichier
        uni_name = filename.replace('.pdf', '').replace('_', ' ')
        
        # Nettoyage
        cleaned_content = clean_text(doc.page_content)
        
        # S'il reste du contenu utile
        if len(cleaned_content) > 50:
            # INJECTION DE CONTEXTE : On ajoute le nom de l'université au début du chunk
            enriched_content = f"Document Source: {uni_name}\n\n{cleaned_content}"
            
            # On met à jour le contenu
            doc.page_content = enriched_content
            processed_docs.append(doc)

    print(f"✅ {len(processed_docs)} pages traitées et enrichies.")

    # 3. Chunking Optimisé
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200,
        chunk_overlap=300,
        separators=["\n\n", "(?<=\. )", "\n", " ", ""]
    )
    
    chunks = text_splitter.split_documents(processed_docs)
    print(f"✂️  Génération de {len(chunks)} fragments (chunks) optimisés.")

    # 4. Embeddings & Indexation
    print(f"🧠 Calcul des vecteurs avec {MODEL_EMBEDDING}...")
    embeddings = HuggingFaceEmbeddings(
        model_name=MODEL_EMBEDDING,
        model_kwargs={'device': DEVICE}
    )

    vectorstore = FAISS.from_documents(chunks, embeddings)
    vectorstore.save_local(DB_FAISS_PATH)
    print(f"✅ Base de données sauvegardée avec succès dans '{DB_FAISS_PATH}'")

if __name__ == "__main__":
    load_and_process_documents()

<>:90: SyntaxWarning: invalid escape sequence '\.'
<>:90: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_15334/899073198.py:90: SyntaxWarning: invalid escape sequence '\.'
  separators=["\n\n", "(?<=\. )", "\n", " ", ""]


--- 🚀 Démarrage du Traitement Avancé (Sur CUDA) ---
📄 6866 pages brutes chargées.
🧹 Nettoyage et Enrichissement des données...
✅ 6796 pages traitées et enrichies.
✂️  Génération de 33923 fragments (chunks) optimisés.
🧠 Calcul des vecteurs avec sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2...


/tmp/ipykernel_15334/899073198.py:98: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


✅ Base de données sauvegardée avec succès dans 'vectorstore/db_faiss'


In [ ]:
import os
import torch
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

# --- CONFIGURATION ---
os.environ["GROQ_API_KEY"]=os.getenv("Groq_API_KEY")
DB_FAISS_PATH = "vectorstore/db_faiss"
MODEL_EMBEDDING = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
MODEL_LLM = "llama-3.3-70b-versatile"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"--- ⚙️ CHARGEMENT DU SYSTÈME MÉMOIRE ({DEVICE.upper()}) ---")

# 1. Embeddings
embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_EMBEDDING,
    model_kwargs={'device': DEVICE}
)

# 2. Vectorstore & Retriever
try:
    vectorstore = FAISS.load_local(DB_FAISS_PATH, embeddings, allow_dangerous_deserialization=True)
    retriever = vectorstore.as_retriever(search_kwargs={'k': 8})
    print("✅ Base chargée.")
except Exception as e:
    print(f"❌ Erreur de chargement : {e}")

# 3. LLM
llm = ChatGroq(temperature=0.0, model_name=MODEL_LLM)

/home/nasser/Projects/Conversational-Chatbot/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- ⚙️ CHARGEMENT DU SYSTÈME MÉMOIRE (CUDA) ---


/tmp/ipykernel_23069/3008227541.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


✅ Base chargée.


In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableBranch

# --- ÉTAPE 1 : LA CHAÎNE DE REFORMULATION (Contextualize) ---
# Ce prompt sert uniquement à réécrire la question en fonction de l'historique
contextualize_q_system_prompt = """
Given a chat history and the latest user question which might reference context in the chat history, 
formulate a standalone question which can be understood without the chat history. 
Do NOT answer the question, just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

# Cette chaîne ne s'active que s'il y a un historique
contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

# --- ÉTAPE 2 : LA CHAÎNE DE RÉPONSE (Answer) ---
qa_system_prompt = """
You are an expert academic advisor for international students.
You have access to official documents from several universities.

INSTRUCTIONS:
1.  **Analyze the Context:** Look at the "Document Source" header in each text chunk.
2.  **Language:** ALWAYS answer in **ENGLISH**.
3.  **Accuracy:** Use the provided context only. If you don't know, say so.
4.  **Format:** Use bullet points for lists.

CONTEXT:
{context}
"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder(variable_name="chat_history"), # On garde l'historique aussi ici au cas où
    ("human", "{question}"),
])

# Fonction pour formater les docs
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# La chaîne finale
qa_chain = qa_prompt | llm | StrOutputParser()

print("✅ Système de mémoire conversationnelle prêt.")

✅ Système de mémoire conversationnelle prêt.


In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
import time

def start_memory_chat():
    # 1. Initialisation de la mémoire pour cette session
    chat_history = []
    
    print("\n" + "="*60)
    print("🤖 MEMORY CHATBOT READY (English Output)")
    print("Example: Ask about a university, then just ask 'What are the fees?'")
    print("="*60 + "\n")

    while True:
        question = input("👉 VOUS : ")
        if question.lower() in ['exit', 'quit', 'q']:
            print("Goodbye!")
            break
        
        start_time = time.time()
        
        try:
            # --- PHASE A : REFORMULATION DE LA QUESTION ---
            if chat_history:
                # Si on a de la mémoire, on demande au LLM de préciser la question
                print("🔄 Reformulation de la question avec le contexte...")
                reformulated_question = contextualize_q_chain.invoke({
                    "chat_history": chat_history,
                    "question": question
                })
                print(f"   (Question interne : '{reformulated_question}')")
            else:
                reformulated_question = question

            # --- PHASE B : RECHERCHE (Retrieval) ---
            # On cherche avec la question REFORMULÉE (plus précise)
            print("🔍 Recherche dans les documents...")
            retrieved_docs = retriever.invoke(reformulated_question)
            
            # --- PHASE C : GÉNÉRATION ---
            print("🧠 Génération de la réponse...")
            response = qa_chain.invoke({
                "context": format_docs(retrieved_docs),
                "chat_history": chat_history,
                "question": reformulated_question
            })
            
            # --- PHASE D : MISE À JOUR MÉMOIRE ---
            # On stocke l'échange pour le prochain tour
            chat_history.extend([
                HumanMessage(content=question),
                AIMessage(content=response)
            ])
            
            # Affichage
            end_time = time.time()
            sources = set([d.metadata.get('source', '').split('/')[-1] for d in retrieved_docs])
            
            print("\n" + "-"*60)
            print(f"⏱️ {end_time - start_time:.2f}s | Sources: {', '.join(sources)}")
            print("-" * 60)
            print(f"💡 ANSWER:\n{response}")
            print("="*60 + "\n")

        except Exception as e:
            print(f"❌ Erreur : {e}")

if __name__ == "__main__":
    start_memory_chat()


🤖 MEMORY CHATBOT READY (English Output)
Example: Ask about a university, then just ask 'What are the fees?'

🔍 Recherche dans les documents...
🧠 Génération de la réponse...

------------------------------------------------------------
⏱️ 2.31s | Sources: HASSAN2_UNIVERCITY.pdf
------------------------------------------------------------
💡 ANSWER:
It seems like you provided a large document in French and Arabic, which appears to be an activity report from the University Hassan II of Casablanca (UH2C) for the year 2019-2020. 

To provide a helpful response, I would like to know what specific information you are looking for in this document. Are you interested in:

* International partnerships and cooperation?
* Campus life and student activities?
* Academic programs and research?
* University governance and quality assurance?
* Something else?

Please let me know, and I'll do my best to provide a helpful and accurate response based on the provided document. 

Here are some general point